In [ ]:
from PIL import Image
from os import getcwd

from common.utils import * 

%matplotlib inline

input_dir = getcwd() + '/img/input_samples/'

### Cumulus test

In [ ]:
img = Image.open(input_dir + '2.jpg').convert('L')
img -= np.mean(img)
img_fr = find_ft_2d(img)

img_shape = img.shape
y_size, x_size = img_shape
x_shift = x_size // 2
y_shift = y_size // 2
xx = freq_numbers_1d(x_size)
yy = freq_numbers_1d(y_size)
x, y = np.meshgrid(xx, yy)
x_ax_size = 10
y_ax_size = 8.5

magn_raw = np.abs(img_fr)
magn = normalize(magn_raw)
magn_factor = normalize_psd(magn_raw, magn)
phase = np.angle(img_fr)
restored_img = find_ift_2d(magn_factor * magn * np.exp(1j * phase)).real

f, ax = show_images(img, np.log10(magn), phase, restored_img, x_fig_size=x_ax_size, y_fig_size=y_ax_size)

In [ ]:
phase_fr = find_ft_2d(phase)
phase_magn = np.abs(phase_fr)
phase_angle = np.angle(phase_fr)
restored_phase = find_ift_2d(phase_magn * np.exp(1j * phase_angle)).real

f, ax = show_images(np.log10(phase_magn), phase_angle, restored_phase, x_fig_size=x_ax_size, y_fig_size=y_ax_size)

In [ ]:
f_1 = show_surfaces(np.log10(magn), axes=(xx, yy))
# f_1.update_layout(scene = dict(zaxis = dict(range=[-5, 0])))
# f_1.show()

In [ ]:
# f_2 = show_surfaces(phase[x_size // 2 - 10: x_size // 2 + 10, y_size // 2 - 10: y_size // 2 + 10], axes=(list(range(20)), list(range(20))))
f_2 = show_surfaces(phase, axes=(xx, yy))
# f_2.show()

### Generating pseudo random phase trying to mimic same phase distribution along each axis

In [ ]:
new_phase_x = np.zeros(img_shape)
for i in range(x_size):
    new_phase_x[:, i] = surrogates(phase[:, i], 1)

    
new_phase_y = np.zeros(img_shape)
for i in range(y_size):
    new_phase_y[i, :] = surrogates(phase[i, :], 1)
    

# New phase and 1/f kernel
new_phase_sur = new_phase_x + new_phase_y 

# Restoring image using original phase and generated phase
restored_img = find_ift_2d(magn_factor * magn * np.exp(1j * new_phase_sur)).real

f, ax = show_images(new_phase_sur, restored_img, x_fig_size=x_ax_size, y_fig_size=y_ax_size)

In [ ]:
# Restoring image using 1/f kernel for magnitude and new phase 
# Results are equal to regular FFT syntesis

kernel = freq_pink_filter_2d(xx, yy)
kernel_magn_factor = normalize_psd(magn_raw, kernel)
restored_img = find_ift_2d(kernel_magn_factor * kernel * np.exp(1j * new_phase_sur)).real

f, ax = show_images(np.log10(kernel), restored_img, x_fig_size=x_ax_size, y_fig_size=y_ax_size)

### Restoring image using linearly changing phase

In [ ]:
initial_phase = [np.pi / 3, 2 * np.pi / 3, np.pi, np.pi / 2, 3 * np.pi / 2]
initial_phase += [-i for i in initial_phase]
whitenoise = np.random.normal(0, 1, img_shape)
whitenoise_fr = find_ft_2d(whitenoise)
lin_random_phase = np.angle(whitenoise_fr)

width = 5
scale = 0.2
alpha_x = 1.4
alpha_y = 1.
step = 1

for i in range(-width, width + 1, step):
    initial_phase_x = random.choice(initial_phase)
    initial_phase_y = random.choice(initial_phase)
    phase_y = lin_phase(initial_phase_y, initial_phase_y + random.choice(initial_phase), y_size)
    phase_x = lin_phase(initial_phase_x, initial_phase_x + random.choice(initial_phase), x_size)
    lin_random_phase[y_shift + i, :] = phase_x
    lin_random_phase[:, x_shift + i] = phase_y
    

restored_img_fr = magn_factor * magn * np.exp(1j * lin_random_phase)
restored_img = find_ift_2d(restored_img_fr).real

f, ax = show_images(lin_random_phase, restored_img, x_fig_size=x_ax_size, y_fig_size=y_ax_size)

In [ ]:
magn_factor = normalize_psd(magn_raw, kernel)
restored_img_fr = magn_factor * kernel * np.exp(1j * lin_random_phase)
restored_img = find_ift_2d(restored_img_fr).real

f, ax = show_images(lin_random_phase, restored_img, x_fig_size=x_ax_size, y_fig_size=y_ax_size)

### 1D regression

In [ ]:
# Custom regression formulas

# y_log = np.log(y_vals)
# x_log = np.log(1 + x_vals)

# alpha = -2 * np.sum(y_log * x_log) / np.sum(x_log ** 2) 
# alpha

In [ ]:
# Approximation along Y-axis

alpha = 1.
dy = 1
factor_y = [1, 0.2, 0.2, 0.15, 0.25, 0.08]
const_y = [0.0019, 0.0008, 0.0015, 0.001, 0.0012, 0.0008]

y_vals = magn[x_size // 2 + dy, magn.shape[1] // 2:] 
x_vals = np.array(range(len(y_vals)))

w_y = factor_y[abs(dy)] * ((1 + abs(x_vals)) ** (-alpha) + const_y[abs(dy)])
# kernel = freq_pink_filter_2d(xx, yy, factor=1)
# kernel[magn.shape[0] // 2, :] = w

plt.plot(x_vals, w_y)
plt.plot(x_vals[1:], y_vals[1:])
plt.yscale('log')

In [ ]:
# Approximation along X-axis

alpha = 1.45
dx = 1
factor_x = [1, 1, 1, 1, 1, 1]
const_x = [0.00015, 0.00015, 0.00015, 0.00015, 0.00015, 0.00015]

y_vals = magn[magn.shape[0] // 2:, y_size // 2 + dx] 
x_vals = np.array(range(len(y_vals)))
w_x = factor_x[abs(dx)] * ((1 + abs(x_vals)) ** (-alpha) + const_x[abs(dx)])

plt.plot(x_vals, w_x)
plt.plot(x_vals[0:], y_vals[0:])
plt.yscale('log')

In [ ]:
# Approximation along Y = X

alpha1 = 1.8
alpha2 = 1.0

l = magn.shape[0] // 2
y_vals = np.zeros(l)

for i in range(l):
    y_vals[i] = magn[l + i, l + i]
x_vals = np.array(range(len(y_vals))) * 2 ** 0.5

y_log = np.log10(y_vals)
x_log = np.log10(1 + x_vals)

w_diag = (1 + abs(x_vals)) ** (-alpha1) + 0.02 * (1 + abs(x_vals)) ** (-alpha2)
# kernel[:, magn.shape[1] // 2] = w

plt.plot(x_vals, w_diag)
plt.plot(x_vals[1:], y_vals[1:])
plt.yscale('log')

In [ ]:
# Combined kernel with previous approximation results

width = 5
scale = 0.2
alpha_x = 1.4
alpha_y = 1.
step = 1

# kernel = freq_pink_filter_2d(xx, yy, factor=0.5)
kernel = fit_clement(xx, yy, alpha1, alpha2)
# kernel = np.zeros((x_size, y_size))

for i in range(-width, width + 1, step):
    w_y = factor_y[abs(i)] * ((1 + abs(yy)) ** (-alpha_y) + const_y[abs(i)])
    w_x = factor_x[abs(i)] * ((1 + abs(xx)) ** (-alpha_x) + const_x[abs(i)])
    kernel[y_shift + i, :] = w_x
    kernel[:, x_shift + i] = w_y
    
# kernel[magn.shape[0] // 2, :] = w_y 
# kernel[:, magn.shape[1] // 2] = w_x 
kernel = normalize(kernel)

f = show_surfaces(np.log10(kernel), axes=(xx, yy))
# fig.update_layout(scene=dict(zaxis = dict(range=[-5, 0])))
# f.show()

In [ ]:
# Restring image using approximated kernel and original phase
restored_img_1 = find_ift_2d(kernel * np.exp(1j * phase)).real
a, b = lin_regression(restored_img_1, img)
restored_img_1 = a * restored_img_1 + b

# Restring image using approximated kernel and random phase
whitenoise = np.random.normal(0, 1, (y_size, x_size))
random_phase = np.angle(find_ft_2d(whitenoise))
kernel_magn_factor = normalize_psd(magn_raw, kernel)
restored_img_2 = find_ift_2d(kernel_magn_factor * kernel * np.exp(1j * random_phase)).real

# Restring image using approximated kernel and linear phase
restored_img_3 = find_ift_2d(kernel_magn_factor * kernel * np.exp(1j * lin_random_phase)).real

# plt.imshow(restored_img_1, cmap='gray', vmin=np.min(img), vmax=np.max(img))
f, ax = show_images(restored_img_1, restored_img_2, restored_img_3, x_fig_size=x_ax_size, y_fig_size=y_ax_size)

### 2D regression

In [ ]:
y_vals = magn
x_vals = freq_filter_2d(xx, yy)

y_log = np.log10(y_vals)
x_log = np.log10(1 + x_vals)

alpha = -2 * np.sum(y_log * x_log) / np.sum(x_log ** 2) 
kernel = freq_pink_filter_2d(xx, yy, factor=alpha / 2)
print(f"alpha = {alpha / 2}")

f = show_surfaces(np.log10(kernel), np.log10(magn), axes=(xx, yy))
# f.show()

In [ ]:
# Restoring image using approximated kernel and original phase 
kernel_magn_factor = normalize_psd(magn_raw, kernel)
restored_image_1 = find_ift_2d(kernel_magn_factor * kernel * np.exp(1j * phase)).real
a, b = lin_regression(restored_image_1, img)
restored_image_1 = a * restored_image_1 + b

# Restoring image using approximated kernel and random phase 
restored_image_2 = find_ift_2d(kernel_magn_factor * kernel * np.exp(1j * random_phase)).real

# Restoring image using approximated kernel and linear phase 
restored_image_3 = find_ift_2d(kernel_magn_factor * kernel * np.exp(1j * lin_random_phase)).real

f, ax = show_images(restored_image_1, restored_image_2, restored_image_3, vrange=(np.min(img), np.max(img)), 
                    x_fig_size=x_ax_size, y_fig_size=y_ax_size)

### New approach

In [ ]:
x_arr, y_arr = np.meshgrid(xx, yy)
window = normalize(1 - np.sqrt(x_arr ** 2 + y_arr ** 2))
window = window

windowed_img = window * img
window_ft = find_ft_2d(window)
windowed_img_ft = find_ft_2d(windowed_img)
  
magn_windowed = np.abs(windowed_img_ft)
magn_n = normalize(magn_windowed)
magn_l = np.log10(magn_n)
phase_windowed = np.angle(windowed_img_ft)
restored_img = find_ift_2d(magn_windowed * np.exp(1j * phase_windowed)).real

vmin = np.min(windowed_img)
vmax = np.max(windowed_img)

f, ax = show_images(windowed_img, np.log10(magn_windowed), phase_windowed, restored_img, 
                    x_fig_size=x_ax_size, y_fig_size=y_ax_size)

In [ ]:
windowed_img_ft = find_ft_2d(phase_windowed)
m = np.abs(windowed_img_ft)
a = np.angle(windowed_img_ft)

whitenoise = np.random.normal(0, 1, img_shape)
psd = m * m
restored_phase = find_ift_2d(psd ** 0.5 * np.abs(whitenoise) * np.exp(1j * a)).real

kernel = freq_pink_filter_2d(xx, yy, factor=0.5)
cloud_hf = find_ift_2d(magn_windowed * np.exp(1j * restored_phase)).real
a, b = lin_regression(cloud_hf, img)
cloud_hf = a * cloud_hf + b

f, ax = show_images(np.log10(m), restored_phase, cloud_hf, x_fig_size=x_ax_size, y_fig_size=y_ax_size)

In [ ]:
alpha1 = 1.8
alpha2 = 1.0
alpha_x = 1.4
alpha_y = 1.

whitenoise = np.random.normal(0, 1, img_shape)
whitenoise_fr = find_ft_2d(whitenoise)
kernel = fit_clement(xx, yy, alpha1, alpha2, eta=0.5)
cloud_lf = find_ift_2d(whitenoise_fr * kernel).real
a, b = lin_regression(cloud_lf, img)
cloud_lf = a * cloud_lf + b

f, ax = show_images(np.log10(kernel), cloud_hf + 0.001 * cloud_lf, x_fig_size=x_ax_size, y_fig_size=y_ax_size)

### Defining an angle between X-axis and ellipse

In [ ]:
weight_exp = 2
threshold = 9
width_x = 5
width_y = 0

# Removing lines along X-axis to get better angle estimation 
min_magn = np.min(magn_windowed)
fit_magn = np.log(magn_windowed / min_magn)

for w in range(-width_y, width_y):
    fit_magn[:, y_shift + w] = fit_magn[:, y_shift + width_y] * 0

for w in range(-width_x, width_x):
    fit_magn[x_shift + w, :] = fit_magn[x_shift + width_x, :]

# Clipping magnitude 
fit_magn = np.where(fit_magn < threshold, 0, fit_magn)
fit_magn_max = np.exp(np.max(fit_magn))
fit_magn = fit_magn / np.mean(fit_magn)

# Defining slope with regression
avg_xy = -np.sum(x * y * fit_magn ** weight_exp)
avg_y2 = np.sum(y * y * fit_magn ** weight_exp)
avg_z2 = np.sum(fit_magn ** weight_exp)
a = avg_xy / avg_y2
angle = np.arctan(a) * 180 / np.pi

# Defining STD along the lines with the found slopes
lperp2 = np.sum((x + a * y) ** 2 * fit_magn ** weight_exp) / avg_z2
lpara2 = np.sum((a * x - y) ** 2 * fit_magn ** weight_exp) / avg_z2
lperp = lperp2 ** 0.5
lpara = lpara2 ** 0.5

# Outputing results 
print(f'A = {a}')
print(f'Angle = {angle}')
print(f'Slope = {1 / a}')
print(f'l_perp = {lperp}')
print(f'l_para = {lpara}')
print(f'l_perp corrected = {lperp2 ** 0.5}')

# Defining lines with found slopes
y_line = np.linspace(0.1 * y_size, 0.9 * y_size)
x_line = -a * (y_line - y_shift) + x_shift

# Defining gaussian approximation of clipped magnitude
model_magn = np.exp(-(x + a * y) ** 2 / 2 / (lperp2)  -(a * x - y) ** 2 / 2 / (lpara2))
# model_magn = model_magn / np.mean(model_magn)

# Plotting 
f, ax = show_images(fit_magn, model_magn)
ax[0].plot(x_line, y_line)
ax[1].plot(x_line, y_line)
plt.show()

### Regression along lines

In [ ]:
def rmse(arr1, arr2, weight_arr=None, normalize=False):
    if arr1.shape != arr2.shape:
        raise ValueError("Shape mismatch between input arrays.")
    
    if weight_arr is None:
        weight_arr = np.ones(arr1.shape)
    elif arr1.shape != weight_arr.shape:
        raise ValueError("Shape mismatch between input arrays and weight array.")
    
    weight_mean = np.mean(weight_arr) 
    diff = weight_arr * (arr1 - arr2) ** 2
    mse = np.sum(diff) / arr1.size
    mse = mse / weight_mean if normalize else mse
    return np.sqrt(mse)


def print_progress_bar(iteration, total, prefix='', suffix='', decimals = 1, 
                       length = 100, fill = '█', print_end='\r'):
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filled_length = int(length * iteration // total)
    bar = fill * filled_length + '-' * (length - filled_length)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end=print_end)
    # Print New Line on Complete
    if iteration == total: 
        print()
        
        
def gaussian(x_vals, y_vals, std):
    arg = x_vals + y_vals
    exp = np.exp(-(arg ** 2) / 2 / std)
    return exp


def get_slice(arr, rows, cols):
    if len(rows) != len(cols):
        print('Row and column indexes arrays have different sizes')
        return

    arr_cols, arr_rows = arr.shape
    sliced_arr = np.zeros(len(rows))

    for index, (row, col) in enumerate(zip(rows, cols)):
        c_l, c_r = int(col), int(col) + 1
        r_t, r_b = int(row), int(row) + 1
        eta_col, eta_row = (col - c_l), (row - r_t)

        if c_r >= arr_cols:
            c_r = c_r - 1

        if r_b >= arr_rows:
            r_b = r_b - 1

        top = (1 - eta_row) * arr[r_t, c_l] + eta_row * arr[r_t, c_r]
        bottom = (1 - eta_row) * arr[r_b, c_l] + eta_row * arr[r_b, c_r]
        total = (1 - eta_col) * top + eta_col * bottom

        sliced_arr[index] = total

    return sliced_arr


max_line_len = int(max(img_shape) * 2 ** 0.5) + 1

In [ ]:
# Parallel line
para_slope = -1 / a
x_line_para = np.linspace(0, x_size, x_size, endpoint=False)
y_line_para = para_slope * (x_line_para - x_size // 2) + y_size // 2

# Clipping arrays
x_line_para, y_line_para = clip_graph(x_line_para, y_line_para, x_max=x_size, y_max=y_size)

# Making arrays to have the same shape as original array
x_line_min, x_line_max = int(np.min(x_line_para)), int(np.max(x_line_para))
x_line_para = np.linspace(x_line_min, x_line_max, max_line_len, endpoint=False)
y_line_para = para_slope * (x_line_para - x_size // 2) + y_size // 2

# Perpendicular line 
perp_slope = -1 / para_slope
x_line_perp = np.linspace(0, x_size, x_size, endpoint=False)
y_line_perp = perp_slope * (x_line_perp - x_size // 2) + y_size // 2

# Clipping arrays
x_line_perp, y_line_perp = clip_graph(x_line_perp, y_line_perp, x_max=x_size, y_max=y_size)
x_line_min, x_line_max = int(np.min(x_line_perp)) + 1, int(np.max(x_line_perp))
x_line_perp = np.linspace(x_line_min, x_line_max, max_line_len, endpoint=False)
y_line_perp = perp_slope * (x_line_perp - x_size // 2) + y_size // 2

# Plotting
f, ax = show_images(magn_l, cmap='gray', x_fig_size=x_ax_size/2, y_fig_size=y_ax_size/2)
ax[0].plot(x_line_para, y_line_para)
ax[0].plot(x_line_perp, y_line_perp)

In [ ]:
#1D regression along para line ORIGINAL APPROACH 
x_indexes_para = np.around(x_line_para).astype(int)
y_indexes_para = np.around(y_line_para).astype(int)

x_vals_para = np.hypot(x_line_para - x_shift, y_line_para - y_shift)
y_vals_para = magn_n[y_indexes_para, x_indexes_para]

max_val = np.max(y_vals_para)
mid = find_index_by_val(y_vals_para, max_val)
x_vals_para = x_vals_para[mid:]
y_vals_para = y_vals_para[mid:]

# Regression variables
x_log_para = np.log(1 + x_vals_para)
y_log_para = np.log(y_vals_para)
alpha_para = -2 * np.sum(y_log_para * x_log_para) / np.sum(x_log_para ** 2) 
alpha_para = alpha_para / 2 

para_approx = (1 + np.linspace(0, np.max(x_vals_para), x_vals_para.size)) ** (-alpha_para)
# para_approx /= np.max(para_approx)

plt.plot(x_vals_para, y_vals_para)
plt.plot(x_vals_para, para_approx)
plt.yscale('log')

print(f'Mid = {mid}')
print(f'Fade para = {alpha_para}')
print(f'Arr max = {max_val}')

In [ ]:
#1D regression along para line NEW APPROACH 
line_mid = x_line_para.size // 2
x_indexes_para_n = np.around(x_line_para[line_mid:]) #.astype(int)
y_indexes_para_n = np.around(y_line_para[line_mid:]) #.astype(int)

x_vals_para_n = np.hypot(x_line_para - x_shift, y_line_para - y_shift)
x_vals_para_n = x_vals_para_n[line_mid:]
y_vals_para_n = get_slice(magn_n, y_indexes_para_n, x_indexes_para_n) # magn_n[y_indexes_para_n, x_indexes_para_n]

max_val = np.max(y_vals_para_n)
mid = find_index_by_val(y_vals_para_n, max_val)
x_vals_para_n = x_vals_para_n[mid:]
y_vals_para_n = y_vals_para_n[mid:]

# Regression variables
x_log_para_n = np.log(1 + x_vals_para_n)
y_log_para_n = np.log(y_vals_para_n)
alpha_para = -2 * np.sum(y_log_para_n * x_log_para_n) / np.sum(x_log_para_n ** 2) 
alpha_para = alpha_para / 2 

para_approx = (1 + np.linspace(0, np.max(x_vals_para_n), x_vals_para_n.size)) ** (-alpha_para)
# para_approx /= np.max(para_approx)

plt.plot(x_vals_para_n, y_vals_para_n)
plt.plot(x_vals_para_n, para_approx)
plt.yscale('log')

print(f'Mid = {mid}')
print(f'Fade para = {alpha_para}')
print(f'Arr max = {max_val}')

In [ ]:
#1D regression along perp line ORIGINAL APPROACH 
x_indexes_perp = np.around(x_line_perp).astype(int)
y_indexes_perp = np.around(y_line_perp).astype(int)

x_vals_perp = np.hypot(x_line_perp - x_shift, y_line_perp - y_shift)
y_vals_perp = magn_n[y_indexes_perp, x_indexes_perp]

max_val = np.max(y_vals_perp)
mid = find_index_by_val(y_vals_perp, max_val)
x_vals_perp = x_vals_perp[mid:]
y_vals_perp = y_vals_perp[mid:]

# Regression variables
x_log_perp = np.log(1 + x_vals_perp)
y_log_perp = np.log(y_vals_perp)
alpha_perp = -2 * np.sum(y_log_perp * x_log_perp) / np.sum(x_log_perp ** 2) 
alpha_perp = alpha_perp / 2 

perp_approx = (1 + np.linspace(0, np.max(x_vals_perp), x_vals_perp.size)) ** (-alpha_perp)
# perp_approx /= np.max(perp_approx)

plt.plot(x_vals_perp, y_vals_perp)
plt.plot(x_vals_perp, perp_approx)
plt.yscale('log')

print(f'Mid = {mid}')
print(f'Fade pero = {alpha_perp}')
print(f'Arr max = {max_val}')

In [ ]:
line_mid = x_line_perp.size // 2
x_indexes_perp_n = np.around(x_line_perp[line_mid:])
y_indexes_perp_n = np.around(y_line_perp[line_mid:])

x_vals_perp_n = np.hypot(x_line_perp - x_shift, y_line_perp - y_shift)
x_vals_perp_n = x_vals_perp_n[line_mid:]
y_vals_perp_n = get_slice(magn_n, y_indexes_perp_n, x_indexes_perp_n)

max_val = np.max(y_vals_perp_n)
mid = find_index_by_val(y_vals_perp_n, max_val)
x_vals_perp_n = x_vals_perp_n[mid:]
y_vals_perp_n = y_vals_perp_n[mid:]

# Regression variables
x_log_perp = np.log(1 + x_vals_perp_n)
y_log_perp = np.log(y_vals_perp_n)
alpha_perp = -2 * np.sum(y_log_perp * x_log_perp) / np.sum(x_log_perp ** 2) 
alpha_perp = alpha_perp / 2 

perp_approx = (1 + np.linspace(0, np.max(x_vals_perp_n), x_vals_perp_n.size)) ** (-alpha_perp)
# perp_approx /= np.max(perp_approx)

plt.plot(x_vals_perp_n, y_vals_perp_n)
plt.plot(x_vals_perp_n, perp_approx)
plt.yscale('log')

print(f'Mid = {mid}')
print(f'Fade para = {alpha_perp}')
print(f'Arr max = {max_val}')

### Playground

In [ ]:
# Defining optimal alpha and eta values that minimize approximation error 

# Factor that represents how stretched ellipse along its major axis
stretch_factor = (lpara2 / lperp2) ** 0.5
magn_n = normalize(magn_windowed)
magn_l = np.log10(magn_n)
std = np.std(magn_windowed)

print(f'Stretch = {stretch_factor}', f'\nSTD = {std}')

In [ ]:
# Original stretched gaussian

perp_gaussian = np.exp(-(a * x - y) ** 2 / 2 / (lpara2))
para_gaussian = np.exp(-(x + a * y) ** 2 / 2 / (lperp2))

perp_my = np.exp(-(perp_slope * x - y) ** 2 / (2 * lpara2))
para_my = np.exp(-(x + perp_slope * y) ** 2 / (2 * lperp2))

"""
f, ax = show_images(perp_my, para_my, perp_my * para_my, x_fig_size=x_ax_size, y_fig_size=y_ax_size)
ax.flatten()[0].plot(x_line_para, y_line_para)
ax.flatten()[0].plot(x_line_perp, y_line_perp)
ax.flatten()[1].plot(x_line_para, y_line_para)
ax.flatten()[1].plot(x_line_perp, y_line_perp)
plt.show()
"""

print((lpara2 / lperp2) ** 0.5)

In [ ]:
# Recreating stretched gaussian in a different manner 

lperp2_new = np.sum((para_slope * x - y) ** 2 * fit_magn ** weight_exp) / avg_z2
lpara2_new = np.sum((x + para_slope * y) ** 2 * fit_magn ** weight_exp) / avg_z2

perp_my = np.exp(-(x + para_slope * y) ** 2 / 2 / lpara2_new)
para_my = np.exp(-(para_slope * x - y) ** 2 / 2 / lperp2_new)

# f, ax = show_images(perp_my, para_my, perp_my * para_my, x_fig_size=x_ax_size, y_fig_size=y_ax_size)
# plt.plot()

print((lpara2_new / lperp2_new) ** 0.5)

In [ ]:
min_error = 10 ** 10
alpha_steps = 50
eta_steps = 50

angle = para_slope
alphas = list(np.linspace(1, 2, alpha_steps))
etas = list(np.linspace(0, 1, eta_steps))
errors = np.zeros((alpha_steps, eta_steps))

power = 1
g = gaussian(x, para_slope * y, lpara2_new) * gaussian(para_slope * x, -y, lperp2_new)
weight = g ** power

for i, alpha in enumerate(alphas):
    print_progress_bar(i + 1, alpha_steps, prefix = 'Progress:', suffix = 'Complete', length=50)
    for j, eta in enumerate(etas):
        my_fit = np.log10(fit_clement_new(xx, yy, alpha, eta=eta, angle=angle))
        error = rmse(magn_l, my_fit, weight, normalize=True)
        errors[i, j] = error
    
        if min_error > error:
            min_error = error
            min_params = (alpha, eta)
            best_fit = my_fit
    
print(*min_params, min_error)

In [ ]:
# Parameters values that minimizes RMSE vs different weighting fuctions 

# alpha / eta / rmse / weighting function used 
1.530612244897959 0.1836734693877551 0.3170742184535750 - stretch_factor gaussian, lperp2_new, lpara2_new, power 1/16
1.510204081632653 0.3061224489795918 0.3141849150420232 - stretch_factor gaussian, lperp2_new, lpara2_new, power 1/8
1.510204081632653 0.4285714285714286 0.3059434597361637 - stretch_factor gaussian, lperp2_new, lpara2_new, power 1/4
1.489795918367347 0.5102040816326531 0.2980129494208664 - stretch_factor gaussian, lperp2_new, lpara2_new, power 1/2
1.551020408163265 0.0816326530612245 0.3101641380512348 - stretch_factor gaussian, lperp2_new, lpara2_new, power 0
1.489795918367347 0.5510204081632653 0.2963893721718555 - stretch_factor gaussian, lperp2_new, lpara2_new, power 1
1.489795918367347 0.5918367346938775 0.2988267656442404 - stretch_factor gaussian, lperp2_new, lpara2_new, power 2
1.469387755102041 0.6122448979591836 0.3010629882540416 - stretch_factor gaussian, lperp2_new, lpara2_new, power 4
1.448979591836735 0.6530612244897959 0.3014733363273698 - stretch_factor gaussian, lperp2_new, lpara2_new, power 8
1.408163265306123 0.6530612244897959 0.3001330362656289 - stretch_factor gaussian, lperp2_new, lpara2_new, power 16

In [ ]:
min_error = 10 ** 10
eta_steps = 1000

alpha = alpha_perp
angle = para_slope

etas = list(np.linspace(0, 1, eta_steps))
errors = np.zeros((eta_steps))

power = 1
g = gaussian(x, para_slope * y, lpara2_new) * gaussian(para_slope * x, -y, lperp2_new) * 0.5 + 0.5
weight = g ** power

for i, eta in enumerate(etas):
    print_progress_bar(i + 1, eta_steps, prefix = 'Progress:', suffix = 'Complete', length=50)
    my_fit = np.log10(fit_clement_new(xx, yy, alpha, eta=eta, angle=angle))
    error = rmse(magn_l, my_fit, normalize=True)
    errors[i] = error
    
    if min_error > error:
        min_error = error
        min_params = eta
        best_fit = my_fit

print(min_params, min_error)
plt.plot(etas, errors)

In [ ]:
# Parameter ETA values that minimizes RMSE vs different weighting fuctions 

# eta / rmse / weighting function used 
0.2242242242242242 0.3265342020042337 - stretch_factor gaussian, lperp2_new, lpara2_new, power 1/16
0.2922922922922923 0.3186435287660681 - stretch_factor gaussian, lperp2_new, lpara2_new, power 1/8
0.4094094094094094 0.3070339788163856 - stretch_factor gaussian, lperp2_new, lpara2_new, power 1/4
0.5115115115115115 0.2979411915626497 - stretch_factor gaussian, lperp2_new, lpara2_new, power 1/2
0.2402402402402402 0.3283416593400807 - stretch_factor gaussian, lperp2_new, lpara2_new, power 0
0.5625625625625625 0.2963620999748029 - stretch_factor gaussian, lperp2_new, lpara2_new, power 1
0.6026026026026026 0.2988631732452551 - stretch_factor gaussian, lperp2_new, lpara2_new, power 2
0.6526526526526526 0.3027102264219166 - stretch_factor gaussian, lperp2_new, lpara2_new, power 4
0.7047047047047047 0.3074197765466930 - stretch_factor gaussian, lperp2_new, lpara2_new, power 8
0.7507507507507507 0.3159201546790819 - stretch_factor gaussian, lperp2_new, lpara2_new, power 16

0.2442442442442442 0.3280924429943492 - stretch_factor gaussian, lperp2_new, lpara2_new, power 1, *0.5 + 0.5 
0.2402402402402402 0.3283416593400807 - stretch_factor gaussian, lperp2_new, no weight

In [ ]:
f, ax = show_images(magn_l, np.log10(fit_clement_new(xx, yy, alpha, eta=0.2442442442442442, angle=angle)))
ax[-1].plot(x_line_para, y_line_para)

In [ ]:
f = show_surfaces(np.log10(fit_clement_new(xx, yy, alpha, eta=0.2442442442442442, angle=angle)), magn_l, axes=(xx, yy))
f.show()

In [ ]:
# Stretching gaussain along axes
# Define the size of the kernel and standard deviation
kernel_size = 21
sigma = 2.0

# Create a standard 2D Gaussian kernel
x = np.linspace(-kernel_size // 2, kernel_size // 2, kernel_size)
y = np.linspace(-kernel_size // 2, kernel_size // 2, kernel_size)
x, y = np.meshgrid(x, y)
kernel = np.exp(-(x**2 + y**2) / (2.0 * sigma**2))

# Create a scaling factor for stretching along y = x
stretching_factor = 2.0  # Adjust this value to control the stretching
new_x = stretching_factor * x

# Create the stretched Gaussian kernel
stretched_kernel = np.exp(-(new_x**2 + y**2) / (2.0 * sigma**2))

# Display the original and stretched kernels
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(kernel, cmap='viridis')
plt.title('Original Gaussian Kernel')

plt.subplot(1, 2, 2)
plt.imshow(stretched_kernel, cmap='viridis')
plt.title('Stretched Gaussian Kernel')
plt.show()

In [ ]:
# Some experiments
# Blending whitenoise phase with original phase

whitenoise = np.random.normal(0, 1, (x_size, y_size))
whitenoise_fr = find_ft_2d(whitenoise)
new_angle = np.angle(whitenoise_fr)

width = 25 #img.shape[0] // 2 - 1
step = 1

for i in range(-width, width + 1, step):
    new_angle[img.shape[0] // 2 + i, :] = phase_windowed[img.shape[0] // 2 + i, :]
    
for j in range(-width, width + 1, step):
    new_angle[:, img.shape[1] // 2 + j] = phase_windowed[:, img.shape[1] // 2 + j]
        
        
dunno = find_ift_2d(magn_windowed * np.exp(1j * new_angle)).real 
# f, ax = show_images(dunno / window)

# sharp_filter = 1 - (freq_sharp_round_filter_2d(xx, yy, radius=900) - freq_sharp_round_filter_2d(xx, yy, radius=50))
# dunno = find_ift_2d(sharp_filter * magn_windowed * np.exp(1j * phase_windowed)).real
dunno = find_ift_2d(magn_windowed * np.exp(1j * new_angle)).real
a, b = lin_regression(dunno, img)
dunno = a * dunno + b

f, ax = show_images(new_angle, dunno)